# TF Text를 이용한 BERT 전처리

## 1. 개요

텍스트 전처리는 end-to-end 방식의 변환으로 정제되지 않은 텍스트를 모델의 정수 input에 맞게 변환하는 과정을 말합니다. 자연어 처리(NLP) 모델에서는 파이썬 코드 수백개의 줄로 전처리를 표현합니다. 텍스트 전처리는 아래와 같은 이유로 생각해야 될 부분이 많습니다.

+ Training-serving skew : 모델의 input 전처리 로직이 모델의 모든 개발 단계(사전 훈련, 미세조정, 평가, 추론 등)에서 일관적으로 사용될 수 있는지를 확신하는 것이 점점 어려워지고 있습니다. 각 개발 단계마다 다른 하이퍼파라미터, 토크나이저, 문자열 전처리 알고리즘 또는 간단한 패키지 모델 input을 이용하는 것은 후에 디버깅을 어렵게 하고 모델에 좋지 않은 영향을 미칠 것입니다.

+ 효율성과 유연성 : 전처리가 오프라인에서 진행되면 추가적인 파일을 읽고 쓰는 비용이 발생합니다. 오프라인 전처리는 동적으로 작동해야하는 전처리가 있을 때 매우 불편합니다. 다른 옵션으로 다시 실험할 때 데이터셋을 다시 일반화해야할 수도 있습니다.

+ 복잡한 모델 인터페이스 : 텍스트 모델은 input이 순수한 텍스트일 때 더 이해하기 쉽습니다. 추가적인 인코딩 단계를 거친 input은 모델을 이해하는데 힘들게 합니다. 전처리의 복잡도를 줄이는 것은 특히 모델 디버깅, 서빙, 평가에 훨씬 도움이 될 것입니다.

추가적으로 간단한 모델 인터페이스일수록 다양한 데이터셋을 모델의 input으로 넣어보기가 편리합니다.

## 2. TF.Text를 이용한 텍스트 전처리

TF.Text 텍스트 전처리 API를 이용하면, 유저의 텍스트 데이터셋을 모델의 정수 input으로 변환할 수 있는 함수를 생성할 수 있습니다.

이번 튜토리얼에서는 TF.Text 전처리 오퍼레이션을 어떻게 사용하고 텍스트 데이터를 BERT 모델의 정수형 input으로 어떻게 변환하는지, BERT의 "Masked LM and Masking Precedure"에 설명된 언어 마스킹 사전 훈련 작업을 위한 입력으로 변환하는 방법을 보여줄 것입니다. 이 프로세스는 텍스트를 하위 단어 단위로 토큰화하고, 문장을 결합하고, 결합한 콘텐츠를 고정된 크기로 자르고, 마스킹 된 언어 모델링 작업을 위한 레이블을 추출하는 작업이 포함됩니다.

### 설정

먼저 필요한 패키지와 라이브러리를 가져오겠습니다.

In [1]:
!pip install -q -U tensorflow-text

In [2]:
import tensorflow as tf
import tensorflow_text as text
import functools

데이터에는 두 개의 텍스트 정보가 포함되어 있으며 `tf.data.Dataset` example을 만들 수 있습니다. 우리의 목표는 훈련에 사용될 `Dataset.map()`을 지원할 함수를 만드는 것입니다.

In [3]:
examples = {
    "text_a": [
               b"Sponge bob Squarepants is an Avenger",
               b"Marvel Avengers"
    ],
    "text_b": [
               b"Barack Obama is the President.",
               b"President is the highest office"
    ]
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))

{'text_a': <tf.Tensor: shape=(), dtype=string, numpy=b'Sponge bob Squarepants is an Avenger'>,
 'text_b': <tf.Tensor: shape=(), dtype=string, numpy=b'Barack Obama is the President.'>}

### 토큰화

우리 첫번째 단계는 데이터세트를 토큰화하는 것입니다. Wordpiece 알고리즘으로 부터 생성된 어휘들을 고려한 BERT 모델을 위한 단어들로 문장을 토큰화하는 `text.BertTokenizer`를 사용할 것입니다.

어휘는 BERT 체크포인트에서 가져올 수도 있고, 데이터에 직접 생성할 수도 있습니다. 이번에는 직접 생성해보겠습니다.

In [4]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5

_VOCAB_SIZE = len(_VOCAB)

# 어휘 딕셔너리 생성
lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=_VOCAB,
        key_dtype=tf.string,
        values=tf.range(
            tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64
        ),
        value_dtype=tf.int64
    ),
    num_oov_buckets=1
)


위의 어휘들을 이용해 `text.BertTokenizer`를 만들고 텍스트 input을 `RaggedTensor`로 토큰화하겠습니다.

In [5]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.string)
bert_tokenizer.tokenize(examples["text_a"])

<tf.RaggedTensor [[[b'Sp', b'##onge'], [b'bob'], [b'Sq', b'##uare', b'##pants'], [b'is'], [b'an'], [b'A', b'##ven', b'##ger']], [[b'Mar', b'##vel'], [b'A', b'##ven', b'##gers']]]>

In [6]:
bert_tokenizer.tokenize(examples["text_b"])

<tf.RaggedTensor [[[b'Bar', b'##ack'], [b'Ob', b'##ama'], [b'is'], [b'the'], [b'President'], [b'[UNK]']], [[b'President'], [b'is'], [b'the'], [b'highest'], [b'office']]]>

모델은 정수형 input을 원하므로 `token_out_type`을 `tf.int64`로 바꾸겠습니다.

In [7]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.int64)
segment_a = bert_tokenizer.tokenize(examples["text_a"])
segment_a

<tf.RaggedTensor [[[22, 9], [24], [23, 11, 10], [28], [14], [15, 13, 7]], [[18, 12], [15, 13, 8]]]>

In [8]:
segment_b = bert_tokenizer.tokenize(examples["text_b"])
segment_b

<tf.RaggedTensor [[[16, 5], [19, 6], [28], [30], [21], [0]], [[21], [28], [30], [27], [29]]]>

`text.BertTokenizer`는 [batch, num_tokens, num_wordpieces]의 형태를 가진 `RaggedTensor`를 반환합니다. 이번 케이스의 경우 `num_tokens` 차원은 단어의 개수를 뜻하고 지금은 모두 일렬로 나열할 것이기 때문에 필요 없으므로 마지막 2개의 차원을 합쳐서 [batch, num_wordpieces] 형태의 `RaggedTensor`를 반환하겠습니다.

In [9]:
segment_a = segment_a.merge_dims(-2, -1)
segment_a

<tf.RaggedTensor [[22, 9, 24, 23, 11, 10, 28, 14, 15, 13, 7], [18, 12, 15, 13, 8]]>

In [10]:
segment_b = segment_b.merge_dims(-2, -1)
segment_b

<tf.RaggedTensor [[16, 5, 19, 6, 28, 30, 21, 0], [21, 28, 30, 27, 29]]>

### 컨텐츠 트리밍

BERT로 들어가는 메인 input은 연속된 두 문장입니다. 하지만 BERT는 고정된 크기와 형태를 가진 input을 요구합니다.

이때 `text.Trimmer`를 사용하여 컨텐츠를 미리 정해진 크기로 만들어 문제를 해결할 수 있습니다. 다른 알고리즘을 이용해서 보존할 컨텐츠를 선택하는 여러 다른 `text.Trimmer` 타입들이 있습니다. `text.RoundRobinTrimmer`는 각 문장에서 왼쪽부터 단어를 선택해서 총 단어가 `max_seq_length`만큼 선택되도록 합니다.
`text.WaterfallTrimmer`는 마지막 문장의 끝부터 선택합니다.

이번 트리밍에서는 `text.RoundRobinTrimmer`를 사용합니다.



In [11]:
trimmer = text.RoundRobinTrimmer(max_seq_length=[_MAX_SEQ_LEN])
trimmed = trimmer.trim([segment_a, segment_b])
trimmed

[<tf.RaggedTensor [[22, 9, 24, 23], [18, 12, 15, 13]]>,
 <tf.RaggedTensor [[16, 5, 19, 6], [21, 28, 30, 27]]>]

총 8개를 선택해야하고 각 세그먼트마다 문장이 2개씩 있으므로 한 문장에서 4개씩 선택한 모습입니다.

### 세그먼트 결합

세그먼트 트리밍 `RaggedTensor`을 결합해서 단일 `RaggedTensor`를 만듭니다. BERT는 특수 토큰을 사용해서 세그먼트의 시작(`[CLS]`)과 끝(`[SEP]`)을 나타낼 수 있습니다.

`text.combine_segments()`를 사용해서 특수 토큰이 삽입된 결합된 `Tensor`를 얻을 수 있습니다.

In [12]:
segments_combined, segments_ids = text.combine_segments(
    [segment_a, segment_b],
    start_of_sequence_id=_START_TOKEN, end_of_segment_id=_END_TOKEN
)
segments_combined, segments_ids

(<tf.RaggedTensor [[3, 22, 9, 24, 23, 11, 10, 28, 14, 15, 13, 7, 4, 16, 5, 19, 6, 28, 30, 21, 0, 4], [3, 18, 12, 15, 13, 8, 4, 21, 28, 30, 27, 29, 4]]>,
 <tf.RaggedTensor [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]>)

### 마스킹된 언어 모델 작업

이제 기본 input을 얻었으므로, "Masked LM and Masking Procedure" 과정에 필요한 input을 추출할 수 있습니다.

마스킹된 언어 모델 작업에는 생각해야 할 두 가지의 소문제가 있습니다.
1. 마스킹을 위해 어떤 아이템을 선택할 것인가?
2. 할당된 값은 무엇인가?

#### 아이템 선택기 만들기

마스킹을 위해 무작위로 아이템을 선택하는 `text.RandomItemSelector`를 사용합니다. (`max_selections_per_batch`, `selection_rate`, `unselectable_ids`)를 인자로 주고 배치 내에서 무작위로 아이템을 선택하고 선택된 아이템을 boolean으로 반환합니다.

random_selector라는 아이템 선택기를 만듭니다.

In [13]:
random_selector = text.RandomItemSelector(
    max_selections_per_batch=_MAX_PREDICTIONS_PER_BATCH,
    selection_rate=0.2,
    unselectable_ids=[_START_TOKEN, _END_TOKEN, _UNK_TOKEN]
)
selected = random_selector.get_selection_mask(
    segments_combined, axis=1
)
selected

<tf.RaggedTensor [[False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False], [False, False, True, False, False, True, False, False, False, False, False, False, False]]>

#### 마스킹된 값 선택기 만들기

`mask_token_rate`는 아이템을 `[MASK]` 토큰으로 바꿉니다.
```
"my dog is hairy" -> "my dog is [MASK]"
```
`random_token_rate`는 아이템을 무작위의 단어로 바꿉니다.
```
"my dog is hairy" -> "my dos is apple"
```
`1 - mask_token_rate - random_token_rate`는 아이템을 바뀌지 않도록 고정합니다.
```
"my dog is hairy" -> "my dog is hairy"
```

`text.MaskedValulesChooser`는 이 로직을 따라 전처리 함수에 사용될 수 있습니다.
mask_values_chooser라는 마스킹 값 선택기를 만들겠습니다. 

그리고 `MaskValuesChooser`가 어떤 input에 대해 80%의 `mask_token_rate`와 디폴트 `random_token_rate`에 따라 어떤 값을 반환하는지 보여드리겠습니다.

In [14]:
input_ids = tf.ragged.constant([[19, 7, 21, 20, 9, 8], [13, 4, 16, 5], [15, 10, 12, 11, 6]])
mask_values_chooser = text.MaskValuesChooser(_VOCAB_SIZE, _MASK_TOKEN, 0.8)
mask_values_chooser.get_mask_values(input_ids)

<tf.RaggedTensor [[1, 1, 21, 1, 1, 1], [1, 2, 1, 1], [1, 1, 1, 1, 1]]>

각 batch에서 임의로 80%의 단어를 선택해서 `[MASK]` 토큰(1)으로 바꾸고, 20%의 단어를 무작위의 다른 단어로 교체합니다.

#### 마스킹된 언어 모델 작업을 위한 input 생성하기

`RandomItemSelector`와 `text.MaskValuesChooser`가 마스킹을 위한 아이템과 값을 선택하는 것으로 도와줬으므로, `text.mask_language_model()`을 이용해 모든 input들을 합쳐보겠습니다.

In [15]:
masked_token_ids, masked_pos, masked_Im_ids = text.mask_language_model(
    segments_combined,
    item_selector=random_selector, mask_values_chooser=mask_values_chooser
)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


`mask_langugage_model()`에 대해 좀 더 알아보겠습니다. `masked_token_ids`의 output은

In [16]:
masked_token_ids

<tf.RaggedTensor [[3, 22, 9, 1, 23, 1, 10, 28, 14, 15, 13, 7, 4, 1, 1, 19, 6, 28, 30, 21, 0, 4], [3, 1, 1, 15, 13, 8, 4, 21, 28, 30, 27, 29, 4]]>

input은 vocabulary를 이용해 인코딩되어 있습니다. `masked_token_ids`를 vocabulary를 이용해 디코딩하면

In [17]:
tf.gather(_VOCAB, masked_token_ids)

<tf.RaggedTensor [[b'[CLS]', b'Sp', b'##onge', b'[MASK]', b'Sq', b'[MASK]', b'##pants', b'is', b'an', b'A', b'##ven', b'##ger', b'[SEP]', b'[MASK]', b'[MASK]', b'Ob', b'##ama', b'is', b'the', b'President', b'[UNK]', b'[SEP]'], [b'[CLS]', b'[MASK]', b'[MASK]', b'A', b'##ven', b'##gers', b'[SEP]', b'President', b'is', b'the', b'highest', b'office', b'[SEP]']]>

몇몇 단어들은 `[MASK]`, `[RANDOM]`, 다른 단어로 바뀐 것을 볼 수 있습니다.

`masked_pos`는 바뀐 단어들의 인덱스를 반환합니다.

In [18]:
masked_pos

<tf.RaggedTensor [[3, 5, 13, 14], [1, 2]]>

`masked_Im_ids`는 바뀐 토큰의 원래 값을 알려줍니다.

In [19]:
masked_Im_ids

<tf.RaggedTensor [[24, 11, 16, 5], [18, 12]]>

In [20]:
tf.gather(_VOCAB, masked_Im_ids)

<tf.RaggedTensor [[b'bob', b'##uare', b'Bar', b'##ack'], [b'Mar', b'##vel']]>

### 모델 input 패딩하기

이제 마지막 전처리 단계로 input들을 패딩하고 고정된 2차원 텐서로 묶어야합니다. 그리고 패드값을 가리키는 마스크 텐서를 생성해야합니다.

`text.pad_model_inputs()`를 사용해 이 과정을 수행할 수 있습니다.

In [21]:
# 마스킹된 토큰들을 패딩하기
input_word_ids, input_mask = text.pad_model_inputs(
    segments_combined, max_seq_length=_MAX_SEQ_LEN
)
input_type_ids, _ = text.pad_model_inputs(
    segments_combined, max_seq_length=_MAX_SEQ_LEN
)
# 마스킹된 input들을 패딩하기
masked_Im_positions, masked_Im_weights = text.pad_model_inputs(
    masked_token_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH
)
masked_Im_ids, _ = text.pad_model_inputs(
    masked_Im_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH
)

model_inputs = {
    "input_word_ids": input_word_ids,
    "input_mask": input_mask,
    "input_type_ids": input_type_ids,
    "masked_Im_ids": masked_Im_ids,
    "masked_Im_positions": masked_Im_positions,
    "masked_Im_weights": masked_Im_weights
}

model_inputs

{'input_mask': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3, 22,  9, 24, 23, 11, 10, 28],
        [ 3, 18, 12, 15, 13,  8,  4, 21]])>,
 'input_word_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3, 22,  9, 24, 23, 11, 10, 28],
        [ 3, 18, 12, 15, 13,  8,  4, 21]])>,
 'masked_Im_ids': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[24, 11, 16,  5,  0],
        [18, 12,  0,  0,  0]])>,
 'masked_Im_positions': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[ 3, 22,  9,  1, 23],
        [ 3,  1,  1, 15, 13]])>,
 'masked_Im_weights': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])>}

## 3. 복습

In [22]:
def bert_pretrain_preprocess(vocab_table, features):
  # Input is a string Tensor of documents, shape [batch, 1].
  text_a = features["text_a"]
  text_b = features["text_b"]

  # Tokenize segments to shape [num_sentences, (num_words)] each.
  tokenizer = text.BertTokenizer(
      lookup_table,
      token_out_type=tf.int64)
  segments = [tokenizer.tokenize(text).merge_dims(
      1, -1) for text in (text_a, text_b)]

  # Truncate inputs to a maximum length.
  trimmer = text.RoundRobinTrimmer(max_seq_length=8)
  trimmed_segments = trimmer.trim(segments)

  # Combine segments, get segment ids and add special tokens.
  segments_combined, segment_ids = text.combine_segments(
      trimmed_segments,
      start_of_sequence_id=_START_TOKEN,
      end_of_segment_id=_END_TOKEN)

  # Apply dynamic masking task.
  masked_token_ids, masked_lm_positions, masked_lm_ids = (
      text.mask_language_model(
        segments_combined,
        random_selector,
        mask_values_chooser,
      )
  )

  # Prepare and pad combined segment inputs
  input_word_ids, input_mask = text.pad_model_inputs(
    segments_combined, max_seq_length=_MAX_SEQ_LEN)
  input_type_ids, _ = text.pad_model_inputs(
    segments_combined, max_seq_length=_MAX_SEQ_LEN)

  # Prepare and pad masking task inputs
  masked_lm_positions, masked_lm_weights = text.pad_model_inputs(
    masked_token_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
  masked_lm_ids, _ = text.pad_model_inputs(
    masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)

  model_inputs = {
      "input_word_ids": input_word_ids,
      "input_mask": input_mask,
      "input_type_ids": input_type_ids,
      "masked_lm_ids": masked_lm_ids,
      "masked_lm_positions": masked_lm_positions,
      "masked_lm_weights": masked_lm_weights,
  }
  return model_inputs

이제 만들어준 `bert_pretrain_preprocess()` 함수를 `Dataset.map()` 안에 넣어줄 것입니다.

In [23]:
dataset = tf.data.Dataset.from_tensors(examples)
dataset = dataset.map(functools.partial(
    bert_pretrain_preprocess, lookup_table
))

next(iter(dataset))

{'input_mask': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3, 22,  9, 24, 23,  4, 16,  5],
        [ 3, 18, 12, 15, 13,  4, 21, 28]])>,
 'input_word_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3, 22,  9, 24, 23,  4, 16,  5],
        [ 3, 18, 12, 15, 13,  4, 21, 28]])>,
 'masked_lm_ids': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[ 9,  5,  0,  0,  0],
        [21, 30,  0,  0,  0]])>,
 'masked_lm_positions': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[ 3, 22,  9, 24, 23],
        [ 3, 18, 12, 15, 13]])>,
 'masked_lm_weights': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])>}